## Introduction to Pandas

### Pandas ("panel data")

The Pandas package is used to explore data, e.g.,
 - To calculate statistical values (mean, max, min etc.)
 - To clean data (remove outliers, account for missing data etc.)
 - To restructure data into a nicer format
 - To prepare data for futher processing, e.g., for classification, clustering, visualization etc.

Its data manipulation syntax resembles that of Numpy array operations. The Pandas package is already installed as part of the Anaconda installation. If this is not your case, you first need to install it on your computer before switching to the exercises.

### What are pandas - series and dataframes

Below is a short example of a 'Series' and 'DataFrame'. We won't worry too much about creating this from scratch since typically, you will use Pandas for reading in data from a given file.

#### Pandas Series
A series is effectively a column of data.

In [ ]:
import pandas as pd   # use the Pandas package and refer to it as 'pd' to save writing out

data_Series = [12, 8, 10, 11]

attendance = pd.Series(data_Series)

print(attendance)

We could make the labelling a bit more helpful.

In [ ]:
# Add some row labels

attendance_labelled = pd.Series(data_Series, index = ['week_1', 'week_2', 'week_3', 'week_4'])

print(attendance_labelled)

#### Pandas DataFrame

A DataFrame is simply a collection of multiple Series.

In [ ]:
import pandas as pd   

data_df = {'Python' : [12, 8, 10, 11], 'Web101' : [6, 10, 8, 11], 'AI' : [11, 7, 6, 9]}

profiles = pd.DataFrame(data_df, index = ['week_1', 'week_2', 'week_3', 'week_4'])

print(profiles)

Note that if you don't use "print", the DataFrame will be printed as a table.

In [ ]:
profiles

### Reading in data

We will use a local database to give you a more realistic picture of what to expect when working on a typical assignment with data manipulation, but also to promote interesting datasets. The database was chosen on the basis that it is quite small, so it is convenient for your first exercises with Pandas.
More interesting databases can be found here: https://opendata.bristol.gov.uk/pages/homepage/

Let us first read the data from the file and see what type it is.

In [ ]:
# Note that all the import statements should go at the top of a script
import os
import pandas as pd

input_file = os.path.join(os.path.abspath(''), 'datafiles', 'public-toilets-community.csv')

toilet_data = pd.read_csv(input_file)

print(type(toilet_data))

Since the data is already of type DataFrame, we can easily print it as a table and check what information it contains.

In [ ]:
toilet_data

In [ ]:
toilet_df = pd.DataFrame(toilet_data)
toilet_df

Note that the printed table doesn't contain all rows, but only the first and last 5 out of a total of 107 rows.

Let's say we are only interested in quickly checking the first rows of the dataset. We can use *x.head(n)* to print the first *n* rows, where *x* is the DataFrame variable (by default, n=5).

In [ ]:
toilet_data.head()

In [ ]:
toilet_data.head(2)

Similarly, let's say we are only interested in quickly checking the last rows of the dataset. We can use *x.tail(n)* to print the last *n* rows, where *x* is the DataFrame variable (by default, n=5).

In [ ]:
toilet_data.tail()

In [ ]:
toilet_data.tail(2)

From a first glance at the table, we notice an *'OBJECTID'*, which, according to the dataset documentation seems to be a unique identifier for each location. Hence, it makes sense to use that as our row index. However, we first need to check whether there are any duplicates in the *'OBJECTID'* column; if so, then it won't work as an index.  

In [ ]:
OBJECTID_data = toilet_data['OBJECTID']
OBJECTID_data

In [ ]:
test = OBJECTID_data.duplicated().any() # returns True if any duplicates, False otherwise
print(test)

Great, we have no duplicates! 

Now we can read the DataFrame again, but using the first column as an index column.

In [ ]:
toilet_data = pd.read_csv(input_file, index_col = [0])  # use first column as the index
toilet_data

That's all there is to it! The data is now in a DataFrame and ready to be examined.  Similarly, can easily read in from a JSON or SQL database (for those who are familiar with them).

### Examining the data

Looking at the first few rows wasn't very helpful. There is too much information and it's not easy to see all of it. But we can do it better by running *x.info()*.

In [ ]:
toilet_data.info()

Now we have a nice summary of the database properties.

### Cleaning and manipulating the data

Let's assume that we want to look at the spread of publicly available toilets by postcode, so that we can see which areas need further work and encourage businesses to join the scheme.  We will want to know when the toilets in a certain postcode areas are open and whether they contain female and accessible toilets. So these fields need to be retained, while all the others can be removed. The steps we will follow are:
 - Remove columns that are clearly not necessary.
 - Have a look at data by postcode.
 - Save the cleaned-up data into a file so that all your hard work isn't lost!

First of all, let's see how to remove columns from the table that don't hold any useful information for us. For instance, the *'Ward'* column can be removed with *x.drop()*.

In [ ]:
toilet_data_tmp = toilet_data.drop("Ward", axis = 1) # axis = 1 means we are dropping a column; use axis = 0 to drop rows
toilet_data_tmp.info()

Great! Now column *'Ward'* is no longer appearing in our new dataframe. However:
 - we'd really like to remove many columns at the same time
 - if we create a new dataframe every time we make a change, we will use lots of memory. It would be better to change the existing dataframe - as long as we are careful not to make mistakes when modifying it directly. 

In order to still have access to the original data in case we need to check something in it afterwards, let's use a separate variable for the cleaned-up data. In our case, we can use *toilet_data_tmp*.

**Note: if we want to overwrite the orginal data when removing columns, we need to use *x.drop(..., inplace=True)*.** By default, *x.drop(..., inplace=False)* and the *drop()* function returns a copy of the input without the removed columns. By setting inplace=True, the function doesn't return an output, but overwrites the input.

In [ ]:
toilet_data_tmp.drop(["Changing Place", "Name","Address", "Automatic opening door", "Pull cord monitored (Y/N/Times)","Male (no. of)",
              "Urinals (no. of)","Radar Key","Unisex (no.of)","Accessibility info",
              "Baby change", "Family Toilet", "Automatic Public Convenience", "Attended", "Full Time Staffing", 
              "geo_shape", "geo_point_2d"], axis = 1, inplace = True)

In [ ]:
# Let's look at the result
toilet_data_tmp.info()

We were quite drastic when removing columns. In real life, you I might want to look a bit more carefully at some of the columns before deleting them in real life. For instance, if you by mistake want to remove again the same column or put the wrong column name, the above will give you an error. 

### Save data to file

Now we will save the modified table to file.

In [ ]:
output_file = os.path.join(os.path.abspath(''), 'datafiles', 'dataset_acc.csv')
toilet_data_tmp.to_csv(output_file)

Now let's open the file and check the data inside.

In [ ]:
input_file = os.path.join(os.path.abspath(''), 'datafiles', 'dataset_acc.csv')

new_data = pd.read_csv(input_file)

new_data

Let us know set the *'OBJECTID'* column (column 0) as the index column.

In [ ]:
new_data = pd.read_csv(input_file, index_col = [0])

new_data

### Additional DataFrame operations

Let's have a look at some of the functions that are available in Pandas.

#### x.describe()

According to the Pandas documentation, *x.describe()* generates descriptive statistics, e.g., a dataset’s distribution (excluding NaN values). By default only numeric fields are returned.

In [ ]:
new_data.describe()

 - top is the most common value in the dataset 
 - freq is the frequency of the most common value

We would expect a few more characteristics to be shown as the output of the *describe()* function. For instance, the mean, 25% percentile, min and max for numeric columns are completely missing. This can be explained by the fact that if we check *x.info()*, we see that some of the columns are of type 'object' and this is a sign that they contain mixed types, e.g., integer and strings.

In [ ]:
new_data.info() # as before

#### x.unique(), x.value_counts(), x.sum()

Let's have a closer look - first at the female column.

In [ ]:
new_data['Female (no. of)'].unique() # gives the number of unique values in the column (aka Pandas Series)

We can also check the frequency with which each of these occur.

In [ ]:
new_data['Female (no. of)'].value_counts() # gives the frequency of each value

In [ ]:
new_data.isna().sum() # gives the total instances of either blank, None or NaN

So, putting these together, I can deduce that for the *'Female'* column:
- there are 107 rows, of which 25 are either blank or NaN, and 87 have values
- the values are actually being held as strings (presumably why the *describe()* function didn't return a wealth of information!
- there is 1 value each for 'No' and 'Yes'

#### NaNs: x.fillna(), x.dropna()

First, let's tackle the NaNs. We don't know why they are NaN as we don't know about the source of the data. 

Here, you have mostly 2 options: 
- ignore completely the entries with NaNs
- overwrite them with a 'special' value, e.g., a very large integer value, which is larger than the current max or something else that makes sense

In this example, we will simply overwrite them with '99'.

In [ ]:
new_data['Female (no. of)'].fillna(99, inplace = True) # Fills all NaN & blanks with 99.

new_data # Let's have a look and see if the NaN has gone

In [ ]:
new_data['Female (no. of)'].unique()

**Note: can you see that 99 appears now as an integer and not a string? That is because we didn't put it between quotes.** If we wanted it to be a string, we should have used *new_data['Female (no. of)'].fillna('99', inplace = True)*

Ok, it looks promising.  Let's just double-check by looking at the number of NaNs and the frequency with which values appear.

In [ ]:
new_data.isna().sum()

In [ ]:
new_data['Female (no. of)'].value_counts() 

Now let's address the 'Yes' and 'No' and the fact that the apparent integers are actually strings - all in one step.

#### x.map()

The x.map(XYZ) function applies whatever is defined by XYZ to every value in a Pandas series.  Let's use this to convert all 'Yes' and 'No's.  We won't replace the original data (*inplace=True*) this time as it's quite a drastic change to make and it might go wrong and spoil the existing DataFrame, so let's create a new column to capture the modifications.

In [ ]:
def yes_no(x):
    if x == 'Yes':
        x = 1
    elif x == 'No':
        x = 0
    elif type(x) == str:
        x = int(x)
    return x

new_data['Female'] = new_data['Female (no. of)'].map(yes_no) # changes yes to 1 and no to 0.  

Now let's check by looking at the frequency.

In [ ]:
new_data['Female'].value_counts()

Looks good!  Now let's check the DataFrame and the .describe() function to see if the results are any different.

In [ ]:
new_data

In [ ]:
new_data.describe()

This already looks much better, but don't forget that there are 25 values of 99, which is purely illustrative data!

Now we can safely remove the *Female (no. of)* column.

In [ ]:
new_data.drop('Female (no. of)', axis = 1, inplace = True)

Let's do the same for the information on Accessible toilets.  We already know that there are no blanks, so we just need to see how many 'Yes' and 'No's there are.

In [ ]:
new_data['Accessible (no. of)'].value_counts()

We will need to make a few a few adjustments to the previous map() function, because 'Yes' appears in different formats.

In [ ]:
def yes(x):
    if x.lower() == 'yes':
        x = 1
    elif type(x) == str:
        x = int(x)
    return x

new_data['Acc'] = new_data['Accessible (no. of)'].map(yes) # changes yes to 1 and no to 0.  

In [ ]:
new_data['Acc'].value_counts()

Let's check again the entire DataFrame before deleting the redundant column.

In [ ]:
new_data

In [ ]:
new_data.drop('Accessible (no. of)', axis = 1, inplace = True)

In [ ]:
new_data.describe()

### Grouping

Finally, let's look at a Pandas function which allows you to take combinations of rows or columns. For example, a company might have a number of transactions for each customer, and they all need to be added together to look at the total sales distribution by customer.

In this case we want to see how the toilets are distributed around the city, so grouping by *'Postcode'* seems sensible - though probably only based on the first 3 or 4 digits. It will be easier to remove the last 3 digits of the postcode rather than select the first 3 or 4 digits. Remember slicing of strings?

In [ ]:
def last_three(string):
    return string[:-4] # note that I've removed 4 characters so that the whitespace is removed as well

new_data['postcode_short'] = new_data['Postcode'].map(last_three)
new_data

In [ ]:
grouped = new_data.groupby(['postcode_short']).sum()
grouped

The *'Female'* column doesn't look particularly sensible because we used *99* instead of blanks - you can improve on this in the exercises.

You might also want to carry out some sanity checking on the result to make sure that things like count, sum, mean etc. are consistent with the original data, i.e., the code hasn't gone wrong somewhere!